<h2 id="Contents">Contents<a href="#Contents"></a></h2>
        <ol>
        <li><a class="" href="#Sequence-Models">Sequence Models</a></li>
<ol><li><a class="" href="#Why-Sequence-Models?">Why Sequence Models?</a></li>
<li><a class="" href="#RNN">RNN</a></li>
<ol><li><a class="" href="#Forward-Propagation">Forward Propagation</a></li>
<li><a class="" href="#Simplified-RNN">Simplified RNN</a></li>
<li><a class="" href="#Backward-Propagation-(Backpropagation-through-time!)">Backward Propagation (Backpropagation through time!)</a></li>
<li><a class="" href="#Bidirectional-RNN-(BRNN)">Bidirectional RNN (BRNN)</a></li>
<li><a class="" href="#Problem-with-RNN">Problem with RNN</a></li>
<ol><li><a class="" href="#Long-Term-Dependencies">Long-Term Dependencies</a></li>
<li><a class="" href="#Vanishing/Exploding-Gradient">Vanishing/Exploding Gradient</a></li>
</ol></ol><li><a class="" href="#GRU">GRU</a></li>
<ol><li><a class="" href="#Update-Gate">Update Gate</a></li>
<li><a class="" href="#Reset-Gate">Reset Gate</a></li>
<li><a class="" href="#Cell-State-Calculation">Cell State Calculation</a></li>
</ol><li><a class="" href="#LSTM">LSTM</a></li>
<ol><li><a class="" href="#Cell-State">Cell State</a></li>
<li><a class="" href="#Gates">Gates</a></li>
<ol><li><a class="" href="#Forget-Gate">Forget Gate</a></li>
<li><a class="" href="#Input-Gate">Input Gate</a></li>
<li><a class="" href="#Output-Gate">Output Gate</a></li>
</ol><li><a class="" href="#Variants-of-LSTM">Variants of LSTM</a></li>
<ol><li><a class="" href="#Peephole-Connections">Peephole Connections</a></li>
<li><a class="" href="#Coupled-Input-and-Forget-Gates">Coupled Input and Forget Gates</a></li>
<li><a class="" href="#GRU">GRU</a></li>
</ol><li><a class="" href="#References">References</a></li>
</ol>

# Sequence Models

Sequence models are the machine learning models that input or output sequences of data. Sequential data includes text streams, audio clips, video clips, time-series data and etc. Some examples of sequnce models are:
- [Recurrent Neural Networks (RNNs)](#RNN)
- [Gated Recurrent Unit (GRU)](#GRU)
- [Long Short-Term Memory (LSTM)](#LSTM)

There are many types of sequence problems:

![](images/0601.jpeg)

## Why Sequence Models?

Traditional neural networks have some limitations which makes them inefficient for processing sequential data. For example, a network cannot process an input of variable length. This is because the input of a network must be of the same size. Another limitation is that a network cannot remember information from a long time ago. This is because a network has no memory. Furthermore, traditional NN's don't share features learned across different positions of text/sequence (like CNN's do). In order to solve these problems, we need to use a sequence model.

## RNN

![](images/0602.png)

Recurrent neural networks address the issue related to memory. They are networks with loops in them, allowing information to persist. In RNN, output at time $t$ depend not only on the input at time $t$ but also on the outputs from previous times.

### Forward Propagation

 There are three weight matrices here: W<sub>ax</sub>, W<sub>aa</sub>, and W<sub>ya</sub> with shapes:
- W<sub>ax</sub>: (NoOfHiddenNeurons, n<sub>x</sub>)
- W<sub>aa</sub>: (NoOfHiddenNeurons, NoOfHiddenNeurons)
- W<sub>ya</sub>: (n<sub>y</sub>, NoOfHiddenNeurons)

These matrices are shared across all the time step. The weight matrix W<sub>aa</sub> is the memory the RNN is trying to maintain from the previous layers.

The equations for forward propagation are:
- $a^{<t>}=g_1(W_{ax}x^{<t>}+W_{aa}a^{<t-1>}+b_a)$
- $y^{<t>}=g_2(W_{ya}a^{<t>}+b_y)$

Usually, $g_1$ is a tanh function and $g_2$ is a softmax function. $a^0$ is initialized to zero.

![](images/0603.png)

### Simplified RNN

We can clean the RNN equations by using some notations. First, we stack $W_{aa}$ and $W_{ax}$ horizontally into a single matrix $W_a$ as:
- $W_a = [W_{aa} W_{ax}]$

Then, we stack $x^{<t>}$ and $a^{<t-1>}$ vertically into a single vector $a^{<t>}$ as:
- $a^{<t>} = [\frac{a^{<t-1>}}{x^{<t>}}]$

Then, we can rewrite the equations as:
$$a^{<t>} = g_1(W_a a^{<t-1>} + b_a)$$
$$y^{<t>} = g_2(W_y a^{<t>} + b_y)$$

### Backward Propagation (Backpropagation through time!)

In the case of a recurrent neural network, the loss function $\mathcal{L}$ of all time steps is defined based on the loss at every time step as follows:
$$
\mathcal{L} = \sum_{t=1}^{T_y} \mathcal{L}^{<t>}
$$
Where $\mathcal{L}^{<t>}$ is the loss at time step $t$, given by
$$
\mathcal{L}^{<t>} = -y^{<t>}\log{\hat{y}^{<t>}} - (1-y^{<t>})\log{(1-\hat{y}^{<t>})}
$$


Backpropagation is done at each point in time. At timestep $T$, the derivative of the loss $\mathcal{L}$ with respect to weight matrix $W$ is expressed as follows:
$$
\frac{\partial \mathcal{L}}{\partial W} = \sum_{t=1}^{T_y} \frac{\partial \mathcal{L}^{<t>}}{\partial W}
$$
Where $\frac{\partial \mathcal{L}^{<t>}}{\partial W}$ is the derivative of the loss at time step $t$ with respect to weight matrix $W$.

![](images/0604.webp)

>The backpropagation here is called **backpropagation through time** because we pass activation `a` from one sequence element to another like backwards in time.

### Bidirectional RNN (BRNN)

There are many other sequence learning tasks contexts where it’s perfectly fine to condition the prediction at every time step on both the leftward and the rightward context. Consider, for example, part of speech detection. For example, consider the three sentences:
- I am ___.
- I am ___ hungry.
- I am ___ hungry, and I can eat half a pig.

In the first sentence “happy” seems to be a likely candidate. The words “not” and “very” seem plausible in the second sentence, but “not” seems incompatible with the third sentences.

Due to this reason, we need to use a bidirectional RNN. In a bidirectional RNN, we have two RNNs, one that processes the input sequence from left to right and one that processes the input sequence from right to left. The output of the two RNNs are then concatenated and fed to a softmax layer. For the first RNN layer, the first input is $x_1$ and the last input is $x_t$, but for the second RNN layer, the first input is $x_t$ and the last input is $x_1$. To produce the output of this bidirectional RNN layer, we simply concatenate together the corresponding outputs of the two underlying unidirectional RNN layers.



![](images/0605.svg)

Mathematically, we can write the equations as:
$$
\begin{align}
a^{<t>}_{fwd} &= g_1(W_{a}^{fwd} a^{<t-1>}_{fwd} + W_{x}^{fwd} x^{<t>} + b_a^{fwd}) \\
a^{<t>}_{bwd} &= g_1(W_{a}^{bwd} a^{<t+1>}_{bwd} + W_{x}^{bwd} x^{<t>} + b_a^{bwd}) \\
\hat{y}^{<t>} &= g_2(W_{y} a^{<t>}_{fwd} + W_{y} a^{<t>}_{bwd} + b_y)
\end{align}
$$

### Problem with RNN

#### Long-Term Dependencies

Sometimes, we only need to look at recent information to perform the present task. For example, consider a language model trying to predict the next word based on the previous ones. If we are trying to predict the last word in “the clouds are in the sky,” we don’t need any further context – it’s pretty obvious the next word is going to be sky. In such cases, where the gap between the relevant information and the place that it’s needed is small, RNNs can learn to use the past information.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png)

But there are also cases where we need more context. Consider trying to predict the last word in the text “I grew up in France… I speak fluent French.” Recent information suggests that the next word is probably the name of a language, but if we want to narrow down which language, we need the context of France, from further back. It’s entirely possible for the gap between the relevant information and the point where it is needed to become very large.

Unfortunately, as that gap grows, RNNs become unable to learn to connect the information.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png)

#### Vanishing/Exploding Gradient

The vanishing and exploding gradient phenomena are often encountered in the context of RNNs. The reason why they happen is that it is difficult to capture long term dependencies because of multiplicative gradient that can be exponentially decreasing/increasing with respect to the number of layers. One way to counter the problem of exploding gradient is to use gradient clipping.

![](https://stanford.edu/~shervine/teaching/cs-230/illustrations/gradient-clipping-en.png?6c3de441dc56aad634dc1a91accb48f2)


Due to all these problems, other architectures have been proposed to overcome these problems. The most popular one is the Long Short-Term Memory (LSTM) network and Gate Recurrent Unit (GRU) network.

## GRU

![](https://stanford.edu/~shervine/teaching/cs-230/illustrations/gru-ltr.png?00f278f71b4833d32a87ed53d86f251c)

Each layer in GRUs has a new variable $c$ which is the memory cell. It can tell to whether memorize something or not. In GRU's $c^{<t>} = a^{<t>}$. In GRU, we have two gates and the cell state. The cell state is kind of like a conveyor belt. It runs straight down the entire chain, with only some minor linear interactions. It’s very easy for information to just flow along it unchanged. Also, as $c^{<t>} = a^{<t>}$, we no longer have the vanishing gradient problem. See [LSTM](#lstm) for details.

### Update Gate

The update gate acts similar to the forget and input gate of an LSTM. It decides what information to throw away and what new information to add. The calculation of the update gate is as follows:
$$
\Gamma_u^{<t>} = \sigma(W_u [a^{<t-1>}, x^{<t>}] + b_u)
$$

### Reset Gate

The Reset Gate is responsible for the short-term memory of the network. This is used to calculate the candidate value of the new cell state. The calculation performed in this gate is very similar to the above:
$$
\Gamma_r^{<t>} = \sigma(W_r [a^{<t-1>}, x^{<t>}] + b_r)
$$

### Cell State Calculation

Using the above two gates, the candidate cell state is updated as:
$$
\tilde{c}^{<t>} = \tanh(W_c [\Gamma_r c^{<t-1>}, x^{<t>}] + b_c)
$$
The updated cell state is
$$
c^{<t>} = \Gamma_u \tilde{c}^{<t>} + (1-\Gamma_u)c^{<t-1>}
$$

## LSTM

Long Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies. LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn!

LSTMs have the same chain like structure like RNN, but the repeating module has a different structure. Instead of having a single neural network layer, there are four, interacting in a very special way.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

### Cell State

The key to LSTMs is the cell state, the horizontal line running through the top of the diagram.

The cell state is kind of like a conveyor belt. It runs straight down the entire chain, with only some minor linear interactions. It’s very easy for information to just flow along it unchanged.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-C-line.png)

The LSTM does have the ability to remove or add information to the cell state, carefully regulated by structures called gates.

Gates are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation. The sigmoid layer outputs numbers between zero and one, describing how much of each component should be let through. A value of zero means “let nothing through,” while a value of one means “let everything through!” An LSTM has three of these gates, to protect and control the cell state.

### Gates

LSTM has three gates which decide which information to keep and which to discard.. The first part is called Forget gate, the second part is known as the Input gate and the last one is the Output gate.

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2021/03/Screenshot-from-2021-03-16-13-41-03-768x342.png)

#### Forget Gate

The first gate decides what information we’re going to throw away from the cell state. This decision is made by a sigmoid layer called the “forget gate layer.” It looks at $h_{t−1}$ and $x^t$, and outputs a number between 0 and 1 for each number in the cell state $C_{t−1}$. A 1 represents “completely keep this” while a 0 represents “completely get rid of this.” For example, the cell state might include the gender of the present subject, so that the correct pronouns can be used. When we see a new subject, we want to forget the gender of the old subject.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png)

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2021/03/Screenshot-from-2021-03-16-14-46-28.png)

#### Input Gate

This gate decides what new information we’re going to store in the cell state. This has two parts. First, a sigmoid layer called the “input gate layer” decides which values we’ll update. Next, a tanh layer creates a vector of new candidate values, $\tilde{C}_t$, that could be added to the state. For example, we’d want to add the gender of the new subject to the cell state, to replace the old one we’re forgetting.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png)

#### Output Gate

This cell is responsible for updating the old cell state, $C_{t−1}$, into the new cell state $C_t$. We multiply the old state by $f_t$, forgetting the things we decided to forget earlier. Then we add $i_t*\tilde{C}_t$. This is the new candidate values, scaled by how much we decided to update each state value. For example, this is where we’d actually drop the information about the old subject’s gender and add the new information, as we decided in the previous steps.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png)

Finally, we need to decide what we’re going to output. This output will be based on our cell state, but will be a filtered version. First, we run a sigmoid layer which decides what parts of the cell state we’re going to output. Then, we put the cell state through tanh (to push the values to be between −1 and 1) and multiply it by the output of the sigmoid gate, so that we only output the parts we decided to.

For the language model example, since it just saw a subject, it might want to output information relevant to a verb, in case that’s what is coming next. For example, it might output whether the subject is singular or plural, so that we know what form a verb should be conjugated into if that’s what follows next.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png)

![](images/0605.png)

### Variants of LSTM

#### Peephole Connections

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-peepholes.png)

A variant of LSTM is gotten using peephole connections. This means that we let the gate layers look at the cell state. The above diagram adds peepholes to all the gates, but many papers will give some peepholes and not others.

#### Coupled Input and Forget Gates

Another variation is to use coupled forget and input gates. Instead of separately deciding what to forget and what we should add new information to, we make those decisions together. We only forget when we’re going to input something in its place. We only input new values to the state when we forget something older.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-tied.png)

#### GRU

We studied GRU earlier. GRU is nothing but a special type of LSTM. It combines the forget and input gates into a single “update gate.” It also merges the cell state and hidden state, and makes some other changes. 

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-GRU.png)

# References

- [CS 230 - Deep Learning](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks)
- [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- [Sequence Model (Andrew NG)](https://www.coursera.org/learn/nlp-sequence-models)
- [Backpropagation in RNN Explained](https://towardsdatascience.com/backpropagation-in-rnn-explained-bdf853b4e1c2)
- [Introduction to Long Short Term Memory (LSTM)](https://www.analyticsvidhya.com/blog/2021/03/introduction-to-long-short-term-memory-lstm)
- [Illustrated Guide to LSTM’s and GRU’s: A step by step explanation](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21)